In [12]:
import os
######################################################
import csv
import pickle
from copy import deepcopy

In [13]:
def readcounts_to_rpm(readcounts):
    for chrom in readcounts.keys():
        totalread = sum(readcounts[chrom])
        for position in range(len(readcounts[chrom])):
            readcounts[chrom][position]/=float(totalread)
            readcounts[chrom][position]*=1E6
            
def writecountsf(rpm,filepath): 
    import struct
    f2=open(filepath+"keys","w")
    for chrom in rpm.keys():
        f=open(filepath+"genome","wb")
        for position in rpm[chrom]:
            f.write(struct.pack("f",position))
        f.close()
        f2.write(chrom+"\n")
    f2.close()

def countstowig(rpm,filestring):
    import random
    f=open(filestring+".wig","w")
    filestring=filestring.partition("_")[0][-3:]

    random.seed(filestring)
    c1=random.randint(0,255)
    random.seed(filestring+"xxx")
    c2=random.randint(0,255)
    random.seed(filestring+"000")
    c3=random.randint(0,255)

    f.write("track name=tracklabel viewLimits=-5:5 color="+str(c1)+','+str(c2)+','+str(c3)+"\n")
    for chrom in rpm.keys():
        if chrom[0:3]=='chr':
            f.write("fixedStep  chrom="+chrom+"  start=1  step=1\n")
        else:
            f.write("fixedStep  chrom=\""+chrom+"\"  start=1  step=1\n") 


        for position in rpm[chrom]:
            f.write(str(position)+"\n")
    f.close()

In [14]:
def five_end_mapping_opposite_stallreporter(samgen,fasta):
    from BCBio import GFF
    from Bio import Seq
    outputdata1={}
    outputdata2={}
    #######################################################
    with open(fasta,'r') as f:
        sequence = ''
        for line in f:
            if line[0] == '>':
                continue
            sequence = sequence+line
    #######################################################    
    outputdata1['plus']=[0 for x in range(len(sequence))]
    outputdata2['minus']=[0 for x in range(len(sequence))]
    
    for read in samgen:
        if read[0][0] == '@':   
            continue
        if read[1] == '4':      
            continue            
        chrom = read[2]             
        readid = read[0]            
        startp = int(read[3]) - 1   
        seq = Seq.Seq(read[9])      
        length = len(seq)           
        if (read[1] == '16'):
            start = startp + length - 1
            outputdata1['plus'][start] += 1 ##### 
        if (read[1] == '0'):
            start = startp
            outputdata2['minus'][start] += 1 ##### 
    return [outputdata1,outputdata2] 

In [15]:
filepath = ''
fastq_path = filepath+'fastq_3RACE/'
fasta_path = filepath+'StallReporters/'

files = ['ks'+str(x) for x in range(246,250)]+['ks'+str(x) for x in range(254,258)]

# cutadapt
cutadapt_input_extension = '.fastq'
cutadapt_output_extension = '-ctadpt.fastq'
cutadapt_adapter = 'TCCTTGGTGCCCGAGTGNNNNNN'
cutadapt_option = '--minimum-length 11 --discard-untrimmed'
cutadapt_output_extension_3end = '-ctadpt_ctadpt3.fastq'
cutadapt_adapter_3end = 'AGATCGGAAGAGCACACGTC'
cutadapt_option_3end = '--minimum-length 11'

# bowtie
inpath = fastq_path
outpath = filepath
bowtie_input_extension = cutadapt_output_extension_3end

# mapping
mapping = '_5map_opposite'

In [16]:
log_file = filepath+'log_3RACE_cutadapt.txt'
###
logs = open(log_file, 'w')
logs.close()
###

for f in files:
    print f
    ###
    cutadapt_inputfile = fastq_path+f+cutadapt_input_extension
    cutadapt_outputfile = fastq_path+f+cutadapt_output_extension

    command = 'cutadapt -g ^%s %s -o %s %s >> %s' % (
        cutadapt_adapter,
        cutadapt_option, 
        cutadapt_outputfile,
        cutadapt_inputfile,
        log_file
        )
    
    logs = open(log_file, 'a+')
    logs.write('\n\n'+f+'\n')
    logs.close()
    
    os.system(command)

ks246
ks247
ks248
ks249
ks254
ks255
ks256
ks257


In [17]:
log_file = filepath+'log_3RACE_cutadapt_3end.txt'
###
logs = open(log_file, 'w')
logs.close()
###

for f in files:
    print f
    ###
    cutadapt_inputfile = fastq_path+f+cutadapt_output_extension
    cutadapt_outputfile = fastq_path+f+cutadapt_output_extension_3end

    command = 'cutadapt -a %s %s -o %s %s >> %s' % (
        cutadapt_adapter_3end,
        cutadapt_option_3end, 
        cutadapt_outputfile,
        cutadapt_inputfile,
        log_file
        )
    
    logs = open(log_file, 'a+')
    logs.write('\n\n'+f+'\n')
    logs.close()
    
    os.system(command)

ks246
ks247
ks248
ks249
ks254
ks255
ks256
ks257


In [18]:
######################################################

files_reporter = ['ks'+str(x) for x in range(246,250)]
stall_reporter = 'secM'

######################################################

###
path_stallreporter = outpath + 'alignments/StallReporter_'+stall_reporter+'/'
if not os.path.exists(outpath + 'alignments/'):
    os.makedirs(outpath + 'alignments/')
if not os.path.exists(path_stallreporter):
    os.makedirs(path_stallreporter)
###   
log_file = filepath+'log_3RACE_Bowtie_StallReporter_'+stall_reporter+'.txt'
logs = open(log_file, 'w')
logs.close()
###

for fname in files_reporter:
    print fname

    bowtie_command = 'bowtie -v 2 -y -m 3 -a --best --strata -S -p 30 '\
        '--un %s%s --max %s%s --al %s%s %s %s%s %s%s 1>> %s 2>> %s'
 
    bowtie_stallreporter = bowtie_command %(
        path_stallreporter,
        fname+'_nomatch.fastq',
        path_stallreporter,
        fname+'_multi.fastq',
        path_stallreporter,
        fname+'_match.fastq',
        fasta_path+'StallReporter_'+stall_reporter,
        inpath,
        fname + bowtie_input_extension,
        path_stallreporter,
        fname + '_StallReporter_'+stall_reporter+'_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n\n'+fname+'\n'+bowtie_stallreporter+'\n')
    logs.close()
    ###
    os.system(bowtie_stallreporter)           

ks246
ks247
ks248
ks249


In [19]:
fasta = fasta_path+'StallReporter_'+stall_reporter+'.fasta'

out_path=filepath+'density_StallReporter/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for f in files_reporter:
    samgenmain_filename=filepath+'alignments/StallReporter_'+stall_reporter+'/'+f+'_StallReporter_'+stall_reporter+'_match.SAM'
    f_samgenmain=open(samgenmain_filename)
    samgenmain=csv.reader(f_samgenmain,delimiter='\t')

    three_mapped=five_end_mapping_opposite_stallreporter(samgenmain,fasta)
    ###
    pickle_readcounts = out_path+f+stall_reporter+mapping+'_readcounts.pickle'
    with open(pickle_readcounts, 'wb') as readcounts_f:
        pickle.dump(three_mapped,readcounts_f)
    ###
    three_mapped_rpm = deepcopy(three_mapped)
    readcounts_to_rpm(three_mapped_rpm[0])
    #readcounts_to_rpm(three_mapped_rpm[1])
    pickle_rpm = out_path+f+stall_reporter+mapping+'_rpm.pickle'
    with open(pickle_rpm, 'wb') as rpm_f:
        pickle.dump(three_mapped_rpm,rpm_f)

    density_path = out_path+f+stall_reporter+mapping
    writecountsf(three_mapped_rpm[0],density_path+"_plus_")
    countstowig(three_mapped_rpm[0],density_path+"_plus")
    writecountsf(three_mapped_rpm[1],density_path+"_minus_")
    countstowig(three_mapped_rpm[1],density_path+"_minus")

In [20]:
######################################################

files_reporter = ['ks'+str(x) for x in range(254,258)]
stall_reporter = 'EPstop'

######################################################

###
path_stallreporter = outpath + 'alignments/StallReporter_'+stall_reporter+'/'
if not os.path.exists(outpath + 'alignments/'):
    os.makedirs(outpath + 'alignments/')
if not os.path.exists(path_stallreporter):
    os.makedirs(path_stallreporter)
###   
log_file = filepath+'log_3RACE_Bowtie_StallReporter_'+stall_reporter+'.txt'
logs = open(log_file, 'w')
logs.close()
###

for fname in files_reporter:
    print fname

    bowtie_command = 'bowtie -v 2 -y -m 3 -a --best --strata -S -p 30 '\
        '--un %s%s --max %s%s --al %s%s %s %s%s %s%s 1>> %s 2>> %s'
 
    bowtie_stallreporter = bowtie_command %(
        path_stallreporter,
        fname+'_nomatch.fastq',
        path_stallreporter,
        fname+'_multi.fastq',
        path_stallreporter,
        fname+'_match.fastq',
        fasta_path+'StallReporter_'+stall_reporter,
        inpath,
        fname + bowtie_input_extension,
        path_stallreporter,
        fname + '_StallReporter_'+stall_reporter+'_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n\n'+fname+'\n'+bowtie_stallreporter+'\n')
    logs.close()
    ###
    os.system(bowtie_stallreporter)           

ks254
ks255
ks256
ks257


In [21]:
fasta = fasta_path+'StallReporter_'+stall_reporter+'.fasta'

out_path=filepath+'density_StallReporter/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for f in files_reporter:
    samgenmain_filename=filepath+'alignments/StallReporter_'+stall_reporter+'/'+f+'_StallReporter_'+stall_reporter+'_match.SAM'
    f_samgenmain=open(samgenmain_filename)
    samgenmain=csv.reader(f_samgenmain,delimiter='\t')

    three_mapped=five_end_mapping_opposite_stallreporter(samgenmain,fasta)
    ###
    pickle_readcounts = out_path+f+stall_reporter+mapping+'_readcounts.pickle'
    with open(pickle_readcounts, 'wb') as readcounts_f:
        pickle.dump(three_mapped,readcounts_f)
    ###
    three_mapped_rpm = deepcopy(three_mapped)
    readcounts_to_rpm(three_mapped_rpm[0])
    #readcounts_to_rpm(three_mapped_rpm[1])
    pickle_rpm = out_path+f+stall_reporter+mapping+'_rpm.pickle'
    with open(pickle_rpm, 'wb') as rpm_f:
        pickle.dump(three_mapped_rpm,rpm_f)

    density_path = out_path+f+stall_reporter+mapping
    writecountsf(three_mapped_rpm[0],density_path+"_plus_")
    countstowig(three_mapped_rpm[0],density_path+"_plus")
    writecountsf(three_mapped_rpm[1],density_path+"_minus_")
    countstowig(three_mapped_rpm[1],density_path+"_minus")